## Реализуйте класс `Vector`, допускающий следующие операции:
1. Конструктор, который принимает на вход итерируемый объект. Данные должны храниться в виде стандартного объекта `array` (не `numpy.array`!) типа `float64`;

2. Класс должен быть итерируемым (реализовать метод `__iter__`). В частности, должен работать следующий код

```
>>> v1 = Vector([3, 4, 5])
>>> x, y, z = v1
>>> x, y, z
(3.0, 4.0, 5.0)
```

3. Корректную операцию сравнения `==` (реализовать метод `__eq__`). Два объекта будем считать равными, если совпадает длина и равны все элементы

```
>>> v1 = Vector([3, 4, 5])
>>> v2 = Vector([3.0, 4.0, 5.0])
>>> v3 = Vector([3, 4])
>>> v1 == v2
True
>>> v1 == v3
False
```

4. Класс должен уметь возвращать в разумном виде свое текстовое представление (реализовать метод `__repr__`):

```
>>> v1 = Vector([3, 4, 5])
>>> v1
Vector((3, 4, 5))
```
При правильно реализованной операции равенства должен работать следующий код:

```
>>> v2 = eval(repr(v1))
>>> v1 == v2
True
```

5. Класс должен уметь корректно приводить объект к строчному типу, в частности, работать с функцией `print` (реализовать метод `__str__`):

```
>>> v1 = Vector([3, 4, 5])
>>> print(v1)
(3.0, 4.0, 5.0)
```

6. Класс должен уметь возвращать свою длину методом `len` (реализовать `__len__`) и допускать индексацию, в том числе с помощью срезов (реализовать `__getitem__`). При этом срез должен возвращать (как и все стандартные типы в питоне) копию объекта.

```
>>> v1 = Vector([3, 4, 5])
>>> len(v1)
3
>>> v1[1]
4.0
>>> v1[-1]
5.0
>>> v1[3.14]
TypeError: Vector indices must be integers
>>> v2 = v1
>>> v3 = v1[:]
>>> v1 is v2
True
>>> v1 is v3
False
```

7. Операцию получения евклидовой длины и приведение к типу `bool`. При приведении к типу `bool` нулевой вектор приводится к `False`, все остальные вектора к `True` (`__abs__`, `__bool__`):

```
>>> v1 = Vector([3.0, 4.0])
>>> abs(v1)
5.0
>>> v2 = Vector([0.0, 0.0, 0.0])
>>> bool(v1)
False
>>> bool(v2)
True
```

8. Операцию поэлементного сложения векторов (`__add__`) и умножения на константу. Умножать на константу должна быть возможность как справа, так и слева (`__mul__` и `__rmul__`):

```
>>> v1 = Vector([3, 4, 5])
>>> v2 = Vector([6, 7, 8])
>>> v1 + v2
Vector([9.0, 11.0, 13.0])
>>> v1 * 2
Vector([6.0, 8.0, 10.0])
>>> 2 * v1
Vector([6.0, 8.0, 10.0])
>>> v1 + 2
TypeError: unsupported operand type(s) for +: 'Vector' and 'int'
```
Для получения корректной ошибки метод должен возвращать `return NotImplemented`.

9. Векторы длины не больше 4 должны допускать возможность индексации с помощью `v1.x, v1.y, v1.z, v1.t`:

```
>>> v1 = Vector(range(4))
>>> v1.x
0.0
>>> v1.y, v1.z, v1.t
(1.0, 2.0, 3.0)
>>> v2 = Vector(range(3))
>>> v2.x, v2.y, v2.z
(0.0, 1.0, 2.0)
>>> v2.t
AttributeError: 'Vector' object has no attribute 't'
```
Для этого реализуйте метод `__getattr__`, который в случае некорректного атрибута возвращает исключение `raise AttributeError`.

10. Реализуйте возможность использовать вектор в виде ключей словаря. Для этого реализуйте метод хеширования вектора (`__hash__`). Как один из вариантов построения хеша можно реализовать операцию `xor` от хешей всех элементов вектора. Возможно, вам пригодятся для этого функции `functools.reduce` и `operator.xor`, если вам надоело писать циклы `for`:

```
>>> v1 = Vector([0, 1, 2])
>>> v2 = Vector([3, 4, 5])
>>> d = {}
>>> d[v1] = 'abc'
>>> d[v2] = 'def'
>>> d
{Vector([0.0, 1.0, 2.0]): 'abc', Vector([3.0, 4.0, 5.0]): 'def'}
```

# 1. Класс Vector

In [12]:
import array as arr
import math
from functools import reduce
from operator import xor

class Vector(object):
    def __init__(self, coord):
        self.length = len(coord)
        self.coord = arr.array("f", coord)

    def __iter__(self):
        return iter(self.coord)
        
    def __eq__(self, compared):   
        return ((self.length == compared.length) & (self.coord[:] == compared.coord[:]))
    
    def __repr__(self):
        string = "Vector("
        string += (repr(list(self.coord)) + ")")
        return string
    
    def __str__(self):
        return repr(tuple(self.coord))
    
    def __len__(self):
        return len(self.coord)
    
    def __getitem__(self, item):
        if (isinstance(item, int)):
            return self.coord[item]
        elif (isinstance(item, slice)):
            return (self.coord[item.start:item.stop:item.step])
        else:
            raise TypeError('Vector indices must be integers')
            
    def __abs__(self):
        return (math.sqrt(sum(self.coord[i]**2 for i in range (self.length))))
    
    def __bool__(self):
        if (self.__abs__() == 0):
            return True
        else:
            return False
        
    def __add__(self, added):
        ans = []
        if (isinstance(added, Vector)):
            for i in range(self.length):
                ans.append(self.coord[i] + added.coord[i])
            return Vector(ans)
        else:
            return NotImplemented
    
    def __mul__(self, prod):
        ans = []
        if isinstance(prod, Vector):
            for i in range (prod.length):
                ans.append(self * prod.coord[i])
            return Vector(ans)
        elif isinstance(prod, (int, float)):
            for i in range (self.length):
                ans.append(self.coord[i] * prod)
            return Vector(ans)
        else:
            return NotImplemented
    
    def __rmul__(self, prod):
        return self.__mul__(prod)
    
    def __getattr__(self, item):
        #d = dict(('x', 0), ('y', 1), ('z', 2), ('t', 3)
        d = {'x': 0, 'y': 1, 'z': 2, 't': 3}
        if ((self.length - 1) < d[item]):
            raise AttributeError("'Vector' object has no attribute '{}'".format(item))
        elif (self.length > 4):
            raise AttributeError
        else:
            return self.coord[d[item]]
    
    def __hash__(self):
        return reduce(xor, map(hash, self.coord))

### Check __iter__

In [13]:
v1 = Vector([3, 4, 5])
x, y, z = v1
x, y, z
#(3.0, 4.0, 5.0)

(3.0, 4.0, 5.0)

### Check __eq__

In [14]:
v1 = Vector([3, 4, 5])
v2 = Vector([3.0, 4.0, 5.0])
v3 = Vector([3, 4])
v1 == v2
#v1 == v2 -> True

True

In [15]:
v1 == v3
#v1 == v3 -> False

False

### Check __repr__

In [16]:
v1 = Vector([3, 4, 5])
v1
#Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [17]:
v2 = eval(repr(v1))
v1 == v2
#True

True

### Check __str__

In [18]:
v1 = Vector([3, 4, 5])
print(v1)

#(3.0, 4.0, 5.0)

(3.0, 4.0, 5.0)


### Check __getitem__

In [19]:
v1 = Vector([3, 4, 5])
len(v1)
#3

3

In [20]:
v1[1]
#4.0

4.0

In [21]:
v1[-1]
#5.0

5.0

In [22]:
v1[3.14]
#TypeError: Vector indices must be integers

TypeError: Vector indices must be integers

In [23]:
v2 = v1
v3 = v1[:]
v1 is v2
#True

True

In [24]:
v1 is v3
#False

False

### Check __abs__ and __bool__

In [25]:
v1 = Vector([3.0, 4.0])
abs(v1)
#5.0

5.0

In [26]:
v2 = Vector([0.0, 0.0, 0.0])
bool(v1)
#False

False

In [27]:
bool(v2)
#True

True

### Check __add__, __mul__ and __rmul__

In [28]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

#Vector([9.0, 11.0, 13.0])

Vector([9.0, 11.0, 13.0])

In [29]:
v1 * 2
#Vector([6.0, 8.0, 10.0])

Vector([6.0, 8.0, 10.0])

In [30]:
2 * v1
#Vector([6.0, 8.0, 10.0])

Vector([6.0, 8.0, 10.0])

In [31]:
v1 + 2
#TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

### Check __getattr__

In [32]:
v1 = Vector(range(4))
v1.x
#0.0

0.0

In [33]:
v1.y, v1.z, v1.t
#(1.0, 2.0, 3.0)

(1.0, 2.0, 3.0)

In [34]:
v2 = Vector(range(3))
v2.x, v2.y, v2.z
#(0.0, 1.0, 2.0)

(0.0, 1.0, 2.0)

In [35]:
v2.t
#AttributeError: 'Vector' object has no attribute 't'

AttributeError: 'Vector' object has no attribute 't'

### Check __hash__

In [37]:
v1 = Vector([0, 1, 2])
v2 = Vector([3, 4, 5])
d = {}
d[v1] = 'abc'
d[v2] = 'def'
d
#{Vector([0.0, 1.0, 2.0]): 'abc', Vector([3.0, 4.0, 5.0]): 'def'}

{Vector([0.0, 1.0, 2.0]): 'abc', Vector([3.0, 4.0, 5.0]): 'def'}

# 2. Класс Key2StrDict

Реализуйте класс `Key2StrDict`, работающий как словарь, но преобразующий все поступающие ключи в строки. Если ключ отсутствует в словаре, объект должен возвращать число `0`. Для реализации может быть удобно (но не необходимо) отнаследоваться от класса `collections.UserDict`. В этом случае достаточно реализовать методы `__missing__`, `__contains__` и `__setitem__`.

```
>>> d = Key2StrDict([('abc', 2), ((1,), 0)])
>>> data = [2, 3.14, frozenset([1, 2, 2]), (1, 5, 6, 8), 2, 2, frozenset([2, 1, 1])]
>>> for t in data:
...     d[t] += 1
>>> d
{'abc': 2, '(1,)': 0, '2': 3, '3.14': 1, 'frozenset({1, 2})': 2, '(1, 5, 6, 8)': 1}
```

In [106]:
from collections import UserDict 

class Key2StrDict(UserDict): 
    def __missing__(self, key):
        self.data[str(key)] = 0
        return 0

    def __contains__(self, key):
        return key in self.mydict
    
    def __setitem__(self, key, value):
        return super().__setitem__(str(key), value)

In [107]:
d = Key2StrDict([('abc', 2), ((1,), 0)])
data = [2, 3.14, frozenset([1, 2, 2]), (1, 5, 6, 8), 2, 2, frozenset([2, 1, 1])]
for t in data:
    d[str(t)] += 1
d

#{'abc': 2, '(1,)': 0, '2': 3, '3.14': 1, 'frozenset({1, 2})': 2, '(1, 5, 6, 8)': 1}

{'abc': 2, '(1,)': 0, '2': 3, '3.14': 1, 'frozenset({1, 2})': 2, '(1, 5, 6, 8)': 1}

# 3. Измерение времени работы

В этом задании нам предстоит решать задачу поиска объекта в множестве и измерять время работы этой операции. Искать мы будем вещественные числа в множестве вещественных чисел. Для этого

1. Сгенерируйте множество чисел, которые мы будем искать (обозначим его `t`, но в коде вы можете назвать его как хотите) и множество чисел, среди которых мы будем искать (`s`). Размер множества `t` должен быть равен 1,000, а размер множества `s` будет варьироваться. Пересечение множеств `t` и `s` должно состоять ровно из 500 элементов. При этом элементы должны быть случайным образом разбросаны, то есть, например, элементы из пересечение не должны лежать в `s` первыми.

2. Реализуйте поиск с помощью списков. Например, таким образом

```
found = 0
for n in t:
    if n in s:
        found += 1
```

3. Реализуйте поиск с помощью словаря (`dict`). Для создания словаря с ключами без значений стоит использовать метод `fromkeys()`.

4. Реализуйте поиск с помощью множеств (`set`), аналогичный пункту 2.

5. Реализуйте поиск с помощью множеств (`set`), используя пересечения

```
found = len(t & s)
```

6. Варьируя размер множества `s` от $10^3$ до $10^7$ с шагом в 10 раз проведите измерения времени для всех 4 описанных методов поиска. Для каждого из методов измерьте его время работы и во сколько раз произошло замедление метода относительно наименьшего размера (1,000). Времена на создание структур данных считать не нужно, только время поиска. Для измерения времени используйте магическую команду `%timeit` Постройте таблицу с результатами (желательно в `pandas`).

7. Сделайте выводы об асимптотической сложности. Выводы сделайте обязательно, но устно и просто их запомните, не надо писать здесь сочинения, пожалуйста.

In [164]:
import timeit

def List(s, t):
    found = 0
    for n in t:
        if n in s:
            found += 1
    return found

def Dict(s, t):
    t_dict = dict.fromkeys(t)
    s_dict = dict.fromkeys(s)

    found = 0
    for n in t_dict.keys():
        if n in s_dict.keys():
            found += 1
    return found

def Set(s, t):
    found = 0
    for n in t:
        if n in s:
            found += 1
    return found

def Set_Cross(s, t):
    return len(t & s)

time = [[0 for x in range(4)] for y in range(5)]

In [165]:
t = set()
s = set()

for i in range (1000):
    t.add(i)
    if (i < 1000):
        if (i % 2 == 0):
            s.add(i)
        else:
            for j in range (10):
                s.add(-i - j)

In [166]:
List(s, t)

500

In [167]:
time[0][0] = timeit.timeit(lambda: List(s, t), number = 10)
time[0][1] = timeit.timeit(lambda: Dict(s, t), number = 10)
time[0][2] = timeit.timeit(lambda: Set(s, t), number = 10)
time[0][3] = timeit.timeit(lambda: Set_Cross(s, t), number = 10)

In [168]:
t = set()
s = set()

for i in range (10000):
    t.add(i)
    if (i < 1000):
        if (i % 2 == 0):
            s.add(i)
        else:
            for j in range (10):
                s.add(-i - j)

In [169]:
Dict(s, t)

500

In [170]:
time[1][0] = timeit.timeit(lambda: List(s, t), number = 10)
time[1][1] = timeit.timeit(lambda: Dict(s, t), number = 10)
time[1][2] = timeit.timeit(lambda: Set(s, t), number = 10)
time[1][3] = timeit.timeit(lambda: Set_Cross(s, t), number = 10)

In [171]:
t = set()
s = set()

for i in range (100000):
    t.add(i)
    if (i < 1000):
        if (i % 2 == 0):
            s.add(i)
        else:
            for j in range (10):
                s.add(-i - j)

In [172]:
Set(s, t)

500

In [173]:
time[2][0] = timeit.timeit(lambda: List(s, t), number = 10)
time[2][1] = timeit.timeit(lambda: Dict(s, t), number = 10)
time[2][2] = timeit.timeit(lambda: Set(s, t), number = 10)
time[2][3] = timeit.timeit(lambda: Set_Cross(s, t), number = 10)

In [174]:
t = set()
s = set()

for i in range (1000000):
    t.add(i)
    t.add(i)
    if (i < 1000):
        if (i % 2 == 0):
            s.add(i)
        else:
            for j in range (10):
                s.add(-i - j)

In [175]:
Set_Cross(s, t)

500

In [176]:
time[3][0] = timeit.timeit(lambda: List(s, t), number = 10)
time[3][1] = timeit.timeit(lambda: Dict(s, t), number = 10)
time[3][2] = timeit.timeit(lambda: Set(s, t), number = 10)
time[3][3] = timeit.timeit(lambda: Set_Cross(s, t), number = 10)

In [177]:
t = set()
s = set()

for i in range (10000000):
    t.add(i)
    t.add(i)
    if (i < 1000):
        if (i % 2 == 0):
            s.add(i)
        else:
            for j in range (10):
                s.add(-i - j)

In [178]:
Set_Cross(s, t)

500

In [179]:
time[4][0] = timeit.timeit(lambda: List(s, t), number = 10)
time[4][1] = timeit.timeit(lambda: Dict(s, t), number = 10)
time[4][2] = timeit.timeit(lambda: Set(s, t), number = 10)
time[4][3] = timeit.timeit(lambda: Set_Cross(s, t), number = 10)

In [ ]:
[time[0][0], time[0][1], time[0][2], time[0][3], 
                   time[1][0], time[1][1], time[1][2], time[1][3],
                   time[2][0], time[2][1], time[2][2], time[2][3], 
                   time[3][0], time[3][1], time[3][2], time[3][3],
                   time[4][0], time[4][1], time[4][2], time[4][3]]

### Таблица результатов

In [187]:
for i in range(5):
    for j in range (4):
        Time [i] = time[i][j]
df = pd.DataFrame({'Размер множества s': ['1000', '10 000', '100 000', '1 000 000', '1 0000 000'] * 4,
                   'Метод': ['Списки', 'Словарь', 'Множества', 'Мн-вo с исп-ем пересечения'] * 5,
                   'Время работы метода': [time[0][0], time[0][1], time[0][2], time[0][3], 
                   time[1][0], time[1][1], time[1][2], time[1][3],
                   time[2][0], time[2][1], time[2][2], time[2][3], 
                   time[3][0], time[3][1], time[3][2], time[3][3],
                   time[4][0], time[4][1], time[4][2], time[4][3]]
                  })
df.groupby(['Размер множества s', 'Метод']).sum().sort_values(by = ['Размер множества s'])


Время работы метода
Размер множества s Метод                                          
1 000 000          Мн-вo с исп-ем пересечения             0.000469
                   Множества                              4.942380
                   Словарь                                1.212283
                   Списки                                 0.051726
1 0000 000         Мн-вo с исп-ем пересечения             0.000210
                   Множества                              0.416853
                   Словарь                                0.112646
                   Списки                                 0.005492
10 000             Списки                                 3.963394
                   Словарь                                0.004224
                   Множества                              0.006112
                   Мн-вo с исп-ем пересечения             0.000155
100 000            Мн-вo с исп-ем пересечения             0.000470
                   Множества                              0.000762
                   Словарь                               15.549917
                   Списки                                 0.335626
1000               Мн-вo с исп-ем пересечения             0.000160
                   Множества                              0.031836
                   Словарь                                0.018801
                   Списки                                 0.000677

### Вывод: исходя из данных, полученных в таблице, самым эффективным по времени является метод множеств с использованием пересечений

#### Прошу прощения, табличка не очень удобная получилась :((

# 4. Какой-то алгоритм

Реализуйте алгоритм, который вам выдали на С++, на питоне. Пользоваться можно только стандартной библиотекой питона, использовать `numpy` пока нельзя. К реализации обязательно должна прилагаться проверка корректности работы. Например, если вы реализуете $QR$-разложение, то нужно проверить, что
1. $R$ - верхняя треугольная;
2. $Q$ - ортогональная;
3. $A = QR$.

Получившийся код нужно прогнать через профилировщик.

### QR-разложение матриц вращениями Гивенса

### Функции

In [289]:
import math

def Rotation(_n, _i, _j, _c, _s, A):
    for k in range(_n):
        Aik = _c * A[i][k] - _s * A[j][k]
        Ajk = _s * A[i][k] + _c * A[j][k]
        A[i][k] = Aik
        A[j][k] = Ajk
        
def Transpose(_n, Q):
    for i in range(_n):
        for j in range(i, _n):
            t = Q[i][j]
            Q[i][j] = Q[j][i]
            Q[j][i] = t

### main

In [290]:
import random  

n = 3
A = [[0 for x in range(n)] for y in range(n)]
_A = [[0 for x in range(n)] for y in range(n)]
Q = [[0 for x in range(n)] for y in range(n)]
R = [[0 for x in range(n)] for y in range(n)]

# заполнение исходной матрицы А случайными числами
for i in range(n):
    for j in range(n):
        A[i][j] = random.randint(0,10)
print("A =", A) 

# заполнение матрицы матрицы _A (= A)
for i in range(n):
    for j in range(n):
        _A[i][j] = A[i][j]

# заполнение матрицы матрицы Q (= I)
for i in range(n):
    for j in range(n):
        if(i == j):
            Q[i][j] = 1
        else:
            Q[i][j] = 0

# заполнение матрицы матрицы R (= A)
for i in range(n):
    for j in range(n):
        R[i][j] = A[i][j]
        
# вращение матриц 
for i in range(n - 1):
    for j in range((i + 1), n):
        norm = math.sqrt(_A[i][i]**2 + _A[j][i]**2)
        c = +_A[i][i] / norm
        s = -_A[j][i] / norm
        Rotation(n, i, j, c, s, _A)
        Rotation(n, i, j, c, s, Q)
        Rotation(n, i, j, c, s, R)
Transpose(n, Q)

A = [[4, 10, 8], [4, 2, 1], [1, 5, 5]]


### Проверка: Q - унитарная

In [291]:
I = [[0 for x in range(n)] for y in range(n)]
for i in range(n):
    for j in range(n):       
        I[i][j] = sum(Q[i][k] * Q[j][k] for k in range(n))
print(I)

[[0.9999999999999998, 8.326672684688674e-17, 0.0], [8.326672684688674e-17, 0.9999999999999997, 2.7755575615628914e-17], [0.0, 2.7755575615628914e-17, 1.0]]


### Проверка: R - верхняя треугольная

In [292]:
for i in range (1, n):
    for j in range (0, i):
        if (abs(R[i][j]) > 0.000001):
            print('Not triangl')
print('Weeee are the champions!!')

Weeee are the champions!!


### Проверка: A = QR

In [293]:
QR = [[0 for x in range(n)] for y in range(n)]
for i in range(n):
    for j in range(n):       
        QR[i][j] = sum(Q[i][k] * R[k][j] for k in range(n))
        if (abs(A[i][j] - QR[i][j]) > 0.000001):
            print('Not equal!')
print('Great!')

Great!
